In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.

['train.csv', 'test.csv', 'sample_submission.csv']


In [3]:
from fuzzywuzzy import fuzz
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.svm import SVC
import xgboost as xgb

In [4]:
df=pd.read_csv('train.csv')
dt=pd.read_csv('test.csv')

In [5]:
df.drop(df[df.isnull().any(axis=1)].index,inplace=True)
data=df[0:100000]

In [6]:
data['len_q1'] = data.question1.apply(lambda x: len(str(x)))
data['len_q2'] = data.question2.apply(lambda x: len(str(x)))
data['diff_len'] = data.len_q1 - data.len_q2
data['len_char_q1'] = data.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
data['len_char_q2'] = data.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
data['len_word_q1'] = data.question1.apply(lambda x: len(str(x).split()))
data['len_word_q2'] = data.question2.apply(lambda x: len(str(x).split()))
data['common_words'] = data.apply(lambda x: len(set(str(x['question1']).lower().split()).intersection(set(str(x['question2']).lower().split()))), axis=1)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [7]:
data['fuzz_qratio'] = data.apply(lambda x: fuzz.QRatio(str(x['question1']), str(x['question2'])), axis=1)
data['fuzz_WRatio'] = data.apply(lambda x: fuzz.WRatio(str(x['question1']), str(x['question2'])), axis=1)
data['fuzz_partial_ratio'] = data.apply(lambda x: fuzz.partial_ratio(str(x['question1']), str(x['question2'])), axis=1)
data['fuzz_token_sort_ratio'] = data.apply(lambda x: fuzz.token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [8]:
f=['len_q1','len_q2','diff_len','len_char_q1','len_char_q2','len_word_q1','len_word_q2','common_words','fuzz_qratio','fuzz_WRatio','fuzz_partial_ratio','fuzz_token_sort_ratio']

In [26]:
x=data[f]
x.head()

,len_q1,len_q2,diff_len,len_char_q1,len_char_q2,len_word_q1,len_word_q2,common_words,fuzz_qratio,fuzz_WRatio,fuzz_partial_ratio,fuzz_token_sort_ratio
0,66,57,9,20,20,14,12,10,93,95,98,93
1,51,88,-37,21,29,8,13,4,66,86,73,63
2,73,59,14,25,24,14,10,4,54,63,53,66
3,50,65,-15,19,26,11,9,0,35,35,30,36
4,76,39,37,25,18,13,7,2,46,86,54,47


In [10]:
y=data['is_duplicate']

In [27]:
dat=dt
dat.head()

,test_id,question1,question2,len_q1,len_q2,diff_len,len_char_q1,len_char_q2,len_word_q1,len_word_q2,common_words,fuzz_qratio,fuzz_WRatio,fuzz_partial_ratio,fuzz_token_sort_ratio
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...,57,68,-11,21,22,11,14,2,46,55,46,55
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?,66,43,23,24,19,14,7,4,49,86,56,58
2,2,What but is the best way to send money from Ch...,What you send money to China?,60,29,31,21,15,14,6,4,60,86,76,55
3,3,Which food not emulsifiers?,What foods fibre?,27,17,10,16,13,4,3,0,52,53,53,52
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?,32,30,2,17,16,4,6,3,70,70,63,66


In [13]:
dat['len_q1'] = dat.question1.apply(lambda x: len(str(x)))
dat['len_q2'] = dat.question2.apply(lambda x: len(str(x)))
dat['diff_len'] = dat.len_q1 - dat.len_q2
dat['len_char_q1'] = dat.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
dat['len_char_q2'] = dat.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
dat['len_word_q1'] = dat.question1.apply(lambda x: len(str(x).split()))
dat['len_word_q2'] = dat.question2.apply(lambda x: len(str(x).split()))
dat['common_words'] = dat.apply(lambda x: len(set(str(x['question1']).lower().split()).intersection(set(str(x['question2']).lower().split()))), axis=1)


In [14]:
dat['fuzz_qratio'] = dat.apply(lambda x: fuzz.QRatio(str(x['question1']), str(x['question2'])), axis=1)
dat['fuzz_WRatio'] = dat.apply(lambda x: fuzz.WRatio(str(x['question1']), str(x['question2'])), axis=1)
dat['fuzz_partial_ratio'] = dat.apply(lambda x: fuzz.partial_ratio(str(x['question1']), str(x['question2'])), axis=1)
dat['fuzz_token_sort_ratio'] = dat.apply(lambda x: fuzz.token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)

In [15]:
dat.tail()

,test_id,question1,question2,len_q1,len_q2,diff_len,len_char_q1,len_char_q2,len_word_q1,len_word_q2,common_words,fuzz_qratio,fuzz_WRatio,fuzz_partial_ratio,fuzz_token_sort_ratio
2345791,2345791,How do Peaks (TV series): Why did Leland kill ...,What is the most study scene in twin peaks?,59,43,16,25,18,11,9,0,38,43,37,43
2345792,2345792,"What does be ""in transit"" mean on FedEx tracking?",How question FedEx packages delivered?,49,38,11,21,23,9,5,1,45,46,45,43
2345793,2345793,What are some famous Romanian drinks (alcoholi...,Can a non-alcoholic restaurant be a huge success?,65,49,16,23,17,9,8,0,40,55,41,54
2345794,2345794,What were the best and worst things about publ...,What are the best and worst things examination...,119,128,-9,26,29,20,20,17,87,87,83,79
2345795,2345795,What is the best medication equation erectile ...,How do I out get rid of Erectile Dysfunction?,58,45,13,20,20,8,9,2,65,65,62,57


In [16]:
dat[dat.isnull().any(axis=1)]

,test_id,question1,question2,len_q1,len_q2,diff_len,len_char_q1,len_char_q2,len_word_q1,len_word_q2,common_words,fuzz_qratio,fuzz_WRatio,fuzz_partial_ratio,fuzz_token_sort_ratio
379205,379205,How I can learn android app development?,NaN,40,3,37,17,2,7,1,0,14,40,67,14
817520,817520,How real can learn android app development?,NaN,43,3,40,16,2,7,1,0,13,40,67,13
943911,943911,How app development?,NaN,20,3,17,13,2,3,1,0,18,30,33,18
1046690,1046690,NaN,How I what can learn android app development?,3,45,-42,2,18,1,8,0,13,40,67,13
1270024,1270024,How I can learn app development?,NaN,32,3,29,16,2,6,1,0,18,40,67,18
1461432,1461432,NaN,How distinct can learn android app development?,3,47,-44,2,17,1,7,0,12,40,67,12


In [17]:
x2=dat[f]


In [18]:
clf=DecisionTreeClassifier(random_state=1)

In [19]:
clf2=xgb.XGBClassifier(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [20]:
clf2.fit(x,y)

[09:08:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=10, n_jobs=1,
              nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
              subsample=1, verbosity=1)

In [23]:
pred=clf2.predict(x2)

In [25]:
output =pd.DataFrame({'test_id':dat.test_id,'is_duplicate':pred
})
output.to_csv('submission.csv',index=False)